In [10]:
#This class will provide the game of RushHour
#ref: https://en.wikipedia.org/wiki/Rush_Hour_(puzzle)
#the parking lot is represented as a 2D grid with empty space has value 0
#any number above 0 is a car. A car that occupies two cells have its number in both the cells

#Actions are specified as a tuple of (car Number, North/East/South/West) eg(1, North)
#The red car to bring out is 1,1
#e.g.
"""
0 0 0 0 0
5 0 0 2 0
5 1 1 2 0
0 0 4 4 4
0 0 3 3 0
"""

#just some alias
North = 0
East  = 1
South = 2
West  = 3
class RushHour :
    def __init__(self,start_config, rows=5,columns=5) : #out_row is the row where the red car is
        self.rows, self.columns = rows, columns
        self.N    = rows * columns
        self.data = tuple(start_config)
        #we fix this now
        self.red_car = 1
        self.out_row = 2 #we can find it out from the data; XXXX
        #we store the horizontal cars and vertical cars seperately
        self.hcars = self._hcars() #dictionary
        self.vcars = self._vcars()
        self.cars  = {}
        self.cars.update(self.hcars)
        self.cars.update(self.vcars)
        
    def is_terminal(self) :
        """checks if the current config is the end or not"""
        #it is end when the redcar reaches the end of the out_row
        exit_index = (self.out_row + 1 ) * self.columns - 1 #we go one ahead and -1 to reach the end of out_row
        if self.data[exit_index] == self.red_car :
            return True
        return False
    
    def actions(self) :
        all_cars = list(self.hcars.keys()) + list(self.vcars.keys())
        return [ (c,a) for c in all_cars for a in [North, East, South, West] if self.can_move((c,a))]
    
    def move(self, action) : #will return new class. Existing one is not changed
        car, act = action
        if act == North :
            start_idx, car_length = self.vcars[car] #we need to the data
            end_idx = start_idx + (car_length-1)*self.columns
            new_start_idx = start_idx - self.columns
            new_data = list(self.data)
            new_data[new_start_idx] = car
            new_data[end_idx] = 0 
            
        elif act == South :
            start_idx, car_length = self.vcars[car] #we need to the data
            end_idx = start_idx + (car_length-1)*self.columns
            
            new_end_idx = end_idx + self.columns
            new_data = list(self.data)
            new_data[new_end_idx] = car
            new_data[start_idx] = 0
            
        elif act == East :
            start_idx, car_length = self.hcars[car]
            end_idx = start_idx + car_length - 1
            
            new_end_idx = end_idx + 1
            new_data = list(self.data)
            new_data[new_end_idx] = car
            new_data[start_idx] = 0
        
        elif act == West :
            start_idx, car_length = self.hcars[car]
            end_idx = start_idx + car_length - 1
            
            new_start_idx = start_idx - 1
            new_data = list(self.data)
            new_data[new_start_idx] = car
            new_data[end_idx] = 0
            
        return RushHour(new_data, self.rows, self.columns)   
    
    def children(self) :
        if self.is_terminal() :
            return []
        #we need to find which all cars can move. We sweep from 0 to end.
        return [ (self.move(a), a) for a in self.actions() ]
    
    def distance(self) :
        return 1 #dummy
        
    def __hash__(self) :
        return hash(self.data)
    
    #-------
    #find the horizontal cars and their positions
    def _hcars(self) : 
        hcars = {} #car_num : position
        for i, car in enumerate(self.data) :
            if i == len(self.data)-1 : #we avoid (i+1)
                break
            if car > 0 and car == self.data[i+1] :
                if car not in hcars :
                    hcars[car] = (i, 2)
                else : #we found this car before
                    idx, car_length = hcars[car]
                    hcars[car] = (idx, car_length+1)
        return hcars
    
    #find the vertical cars and their positions
    def _vcars(self) : 
        vcars = {} #car_num : (position, length)
        for i, car in enumerate(self.data) :
            if i == len(self.data)-self.columns : #we skip the last row
                break
            if car > 0 and car == self.data[i+self.columns] : #is it the same as one below
                if car not in vcars :
                    vcars[car] = (i, 2)
                else : #we found this car before
                    idx, car_length = vcars[car]
                    vcars[car] = (idx, car_length+1)
        return vcars
    
    def can_move(self, action) : #action is a tuple of carnumber and N/E/S/W
        car, act = action
        if car in self.hcars :
            return self._can_move_horizontal(car, act)
        if car in self.vcars :
            return self._can_move_vertical(car, act)
        return False #should not reach here
    
    def _can_move_horizontal(self, car, act) :
        start_idx, car_length = self.hcars[car]
        end_idx = start_idx + car_length - 1
        if act == East : #can we move to right?
            #if not at the right edge and there is space
            if (end_idx+1) % self.columns != 0 and self.data[end_idx+1] == 0: 
                return True
        if act == West : #can we move to left?            
            if start_idx >0 and start_idx % self.columns != 0 and self.data[start_idx-1] == 0: 
                return True
        return False
    
    def _can_move_vertical(self, car, act) :
        return False #XXXX TODO
        start_idx, car_length = self.hcars[car]
        end_idx = start_idx + car_length - 1
        if act == East : #can we move to right?
            #if not at the right edge and there is space
            if (end_idx+1) % self.columns != 0 and self.data[end_idx+1] == 0: 
                return True
        if act == West : #can we move to left?            
            if start_idx >0 and start_idx % self.columns != 0 and self.data[start_idx-1] == 0: 
                return True
        return False

In [11]:
from graphs import str_to_list
s = """
5 0 0 0 0
5 0 0 2 0
5 1 1 2 0
0 0 4 4 4
0 0 3 3 3
"""
r = RushHour(str_to_list(s))

In [20]:
r._vcars()

{5: (0, 3), 2: (8, 2)}

In [12]:
from graphs import a_star_search
path = a_star_search(r)
print(len(path))

KeyboardInterrupt: 

In [17]:
import graphs
help(graphs)

Help on module graphs:

NAME
    graphs - # coding: utf-8

CLASSES
    builtins.object
        Pqueue
    
    class Pqueue(builtins.object)
     |  stores nodes with their weights
     |  updates the weights as the path function goes down the path
     |  retrieves nodes in the order of their weights
     |  
     |  Methods defined here:
     |  
     |  __init__(self, lst)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __len__(self)
     |  
     |  append(self, node, weight)
     |  
     |  pop(self)
     |  
     |  query(self, node)
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)

FUNCTIONS
    a_star_search(start)
        #adjust bfs for weighted graph
    
    connected(graph, star